# Information Retrieval

The process of fetching relevant documents, passages, or data from a large collection based on a query. Traditional IR methods include TF-IDF, BM25, and vector search using embeddings. IR does not generate new content but retrieves existing information.

<img src="../figures/ir.jpg" >

## TF-IDF 
It is a method of information retrieval that is used to rank the importance of words in a document.
It is based on the idea that words that appear in a document more often are more relevant to the document.

**Terminology :**
- t — term (word)
- d — document (set of words)
- N — count of corpus
- corpus — the total document set

**Term Frequency (TF) :** Suppose we have a set of English text documents and wish to rank which document is most relevant to the query , “Data Science is awesome !” A simple way to start out is by eliminating documents that do not contain all three words “Data”,”is”, “Science”, and “awesome”, but this still leaves many documents. To further distinguish them, we might count the number of times each term occurs in each document; the number of times a term occurs in a document is called its term frequency.

tf(t, d) = (Number of times term t appears in document d) / (Total number of terms in document d)


**Document Frequency :**
This measures the importance of document in whole set of corpus, this is very similar to TF. The only difference is that TF is frequency counter for a term t in document d, where as DF is the count of occurrences of term t in the document set N. In other words, DF is the number of documents in which the word is present. We consider one occurrence if the term consists in the document at least once, we do not need to know the number of times the term is present.

df(t) = occurrence of t in N documents

**Inverse Document Frequency(IDF):**
While computing TF, all terms are considered equally important. However it is known that certain terms, such as “is”, “of”, and “that”, may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing IDF, an inverse document frequency factor is incorporated which diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely. IDF is the inverse of the document frequency which measures the informativeness of term t. When we calculate IDF, it will be very low for the most occurring words such as stop words (because stop words such as “is” is present in almost all of the documents, and N/df will give a very low value to that word). This finally gives what we want, a relative weightage.

idf(t) = N/df

Now there are few other problems with the IDF , in case of a large corpus,say 100,000,000 , the IDF value explodes , to avoid the effect we take the log of idf . During the query time, when a word which is not in vocab occurs, the df will be 0. As we cannot divide by 0, we smoothen the value by adding 1 to the denominator. that’s the final formula:

idf(t) = log(N/(df + 1))

**TF-IDF** is a the right measure to evaluate how important a word is to a document in a collection or corpus. here are many different variations of TF-IDF but for now let us concentrate on the this basic version.

tf-idf(t, d) = tf(t, d) * log(N/(df + 1))

In [41]:
#importing the libraries
import numpy as np
from nltk.tokenize import word_tokenize

#sample to consider for TF-IDF
sample_text = [
    'Topic sentences are similar to mini thesis statements.'
    'Like a thesis statement, a topic sentence has a specific '
    'main point. Whereas the thesis is the main point of the essay',
    'the topic sentence is the main point of the paragraph.'
    'Like the thesis statement, a topic sentence has a unifying function. '
    'But a thesis statement or topic sentence alone doesn’t guarantee unity.',
    'An essay is unified if all the paragraphs relate to the thesis,'
    'whereas a paragraph is unified if all the sentences relate to the topic sentence.'
]

In [42]:
#tokenizing the sample text and creating unique set of words
sentences = []
word_set = []

for sent in sample_text:
    words = [word.lower() for word in word_tokenize(sent) if word.isalpha()]
    sentences.append(words)
    for word in words:
        if word not in word_set:
            word_set.append(word)
            
# Set of words
word_set = set(word_set)
# total documents in our corpus
total_docs = len(sample_text)
print('Total documents: ', total_docs)
print('Total words: ', len(word_set))

Total documents:  3
Total words:  35


In [43]:
#Creating index for each word from vocabulary
word_index = {}
for i, word in enumerate(word_set):
    word_index[word] = i

print(word_index)

{'similar': 0, 'guarantee': 1, 'unity': 2, 'or': 3, 'all': 4, 'are': 5, 'sentence': 6, 'has': 7, 'point': 8, 'doesn': 9, 'paragraphs': 10, 'the': 11, 'is': 12, 'an': 13, 'thesis': 14, 'statement': 15, 'specific': 16, 'whereas': 17, 'topic': 18, 'main': 19, 'of': 20, 'essay': 21, 'unifying': 22, 't': 23, 'if': 24, 'a': 25, 'but': 26, 'to': 27, 'sentences': 28, 'relate': 29, 'mini': 30, 'function': 31, 'alone': 32, 'paragraph': 33, 'unified': 34}


In [44]:
#Creating a dictionary for keeping count
def count_dict(sentences):
    count_dict = {}
    for word in word_set:
        count_dict[word] = 0
    for sent in sentences:
        for word in sent:
            count_dict[word] += 1
    return count_dict
    
word_count = count_dict(sentences)
print(word_count)

{'similar': 1, 'guarantee': 1, 'unity': 1, 'or': 1, 'all': 2, 'are': 1, 'sentence': 5, 'has': 2, 'point': 3, 'doesn': 1, 'paragraphs': 1, 'the': 11, 'is': 4, 'an': 1, 'thesis': 6, 'statement': 3, 'specific': 1, 'whereas': 2, 'topic': 6, 'main': 3, 'of': 2, 'essay': 2, 'unifying': 1, 't': 1, 'if': 2, 'a': 7, 'but': 1, 'to': 3, 'sentences': 2, 'relate': 2, 'mini': 1, 'function': 1, 'alone': 1, 'paragraph': 1, 'unified': 2}


In [45]:
def term_frequency(document, word):
    N = len(document)
    occurance = len([token for token in document if token == word])
    return occurance / N

In [46]:
def inverse_document_frequency(word):
    try:
        word_occurance = word_count[word] + 1
    except:
        word_occurance = 1
    return np.log(total_docs / word_occurance)

In [47]:
def tf_idf(sentence):
    vec = np.zeros((len(word_set),))
    for word in sentence:
        tf = term_frequency(sentence, word)
        idf = inverse_document_frequency(word)
        vec[word_index[word]] = tf * idf
    return vec

In [138]:
vectors = []
for sent in sentences:
    vectors.append(tf_idf(sent))

# Create a DataFrame
df = pd.DataFrame(vectors, columns=word_count.keys())
df

,similar,guarantee,unity,or,all,are,sentence,has,point,doesn,...,a,but,to,sentences,relate,mini,function,alone,paragraph,unified
0,0.014481,0.00000,0.00000,0.00000,0.0,0.014481,-0.024755,0.0,-0.020549,0.00000,...,-0.105089,0.00000,-0.010274,0.0,0.0,0.014481,0.00000,0.00000,0.000000,0.0
1,0.000000,0.01308,0.01308,0.01308,0.0,0.000000,-0.067079,0.0,-0.009280,0.01308,...,-0.094919,0.01308,0.000000,0.0,0.0,0.000000,0.01308,0.01308,0.000000,0.0
2,0.000000,0.00000,0.00000,0.00000,0.0,0.000000,-0.026660,0.0,0.000000,0.00000,...,-0.037724,0.00000,-0.022129,0.0,0.0,0.000000,0.00000,0.00000,0.015595,0.0


In [61]:
import numpy as np

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    similarity = dot_product / (norm_a * norm_b)
    return round(similarity, 4)

# Example usage:
similarity_score = cosine_similarity(vectors[0], vectors[1])
print("Cosine Similarity A-B:", similarity_score)
similarity_score = cosine_similarity(vectors[0], vectors[2])
print("Cosine Similarity A-C:", similarity_score)
similarity_score = cosine_similarity(vectors[1], vectors[2])
print("Cosine Similarity B-C:", similarity_score)

Cosine Similarity A-B: 0.9338
Cosine Similarity A-C: 0.8415
Cosine Similarity B-C: 0.8845


Reference
- https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
- https://www.kaggle.com/code/yassinehamdaoui1/creating-tf-idf-model-from-scratch
- https://github.com/ashushekar/TF-IDF-Model/blob/master/tfidf.py
- https://medium.com/@ashwinnaidu1991/creating-a-tf-idf-model-from-scratch-in-python-71047f16494e

## [BM 25 (BM is an abbreviation of best matching)](https://www.cs.otago.ac.nz/homepages/andrew/papers/2014-2.pdf)
It is  a term-based ranking model that aims to provide accurate and relevant search results by scoring documents based on their term frequencies and document lengths. This essay explores the fundamental concepts and working principles of the BM25 ranking algorithm.

$\text{BM25}(D, Q) = \sum_{i=1}^{n} \text{IDF}(q) \cdot \frac{{\text{TF}(q, D) \cdot (k_1 + 1)}}{{\text{TF}(q, D) + k_1 \cdot \left(1 - b + b \cdot \frac{{\lvert D \rvert}}{{\text{avgdl}}}\right)}}$


In [30]:
a = "purple is the best city in the forest".split()
b = "there is an art to getting your way and throwing bananas on to the street is not it".split()
c = "it is not often you find soggy bananas on the street".split()
d = "green should have smelled more tranquil but somehow it just tasted rotten".split()
e = "joyce enjoyed eating pancakes with ketchup".split()
f = "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled".split()
docs = [a, b, c, d, e, f]

In [31]:
import numpy as np

avgdl = sum(len(sentence) for sentence in [a,b,c,d,e,f]) / len(docs)
N = len(docs)

def bm25(word, sentence, k=1.2, b=0.75):
    freq = sentence.count(word)  # or f(q,D) - freq of query in Doc
    tf = (freq * (k + 1)) / (freq + k * (1 - b + b * (len(sentence) / avgdl)))
    N_q = sum([doc.count(word) for doc in docs])  # number of docs that contain the word
    idf = np.log(((N - N_q + 0.5) / (N_q + 0.5)) + 1)
    return round(tf*idf, 4)

In [35]:
bm25('purple', a), bm25('purple', b), bm25('bananas', b), bm25('bananas', c)

(1.7677, 0.0, 0.8425, 1.0543)

### Using Okapi BM25

In [4]:
# ! pip3 install rank_bm25

In [5]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

**Ranking of documents**
Now that we've created our document indexes, we can give it queries and see which documents are the most relevant:

In [6]:
query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
doc_scores
# array([0.        , 0.93729472, 0.        ])

array([0.        , 0.93729472, 0.        ])

In [113]:
docs = bm25.get_top_n(tokenized_query, corpus, n=3)
docs

['It is quite windy in London',
 'How is the weather today?',
 'Hello there good man!']

Reference
- https://medium.com/@evertongomede/understanding-the-bm25-ranking-algorithm-19f6d45c6ce
- https://www.analyticsvidhya.com/blog/2021/05/build-your-own-nlp-based-search-engine-using-bm25/
- https://abishek21.medium.com/building-your-favourite-tv-series-search-engine-information-retrieval-using-bm25-ranking-8e8c54bcdb38

## Appendix

In [71]:
a = "purple is the best city in the forest".split()
b = "there is an art to getting your way and throwing bananas on to the street is not it".split()
c = "it is not often you find soggy bananas on the street".split()

import numpy as np
def tfidf(word):
    tf = []
    count_n = 0
    for sentence in [a, b, c]:
        # calculate TF
        t_count = len([x for x in sentence if word in sentence])
        tf.append(t_count/len(sentence))
        # count number of docs for IDF
        count_n += 1 if word in sentence else 0
    idf = np.log10(len([a, b, c]) / count_n)
    return [round(_tf*idf, 2) for _tf in tf]

tfidf_a, tfidf_b, tfidf_c = tfidf('forest')
print(f"TF-IDF a: {tfidf_a}\nTF-IDF b: {tfidf_b}\nTF-IDF c: {tfidf_c}")

TF-IDF a: 0.48
TF-IDF b: 0.0
TF-IDF c: 0.0


In [107]:
import pandas as pd

first_sentence = "Data Science is the sexiest job of the 21st century".split()
second_sentence = "machine learning is the key for data science".split()
total= set(first_sentence).union(set(second_sentence))
print(total)

wordDictA = dict.fromkeys(total, 0) 
wordDictB = dict.fromkeys(total, 0)
for word in first_sentence:
    wordDictA[word]+=1
    
for word in second_sentence:
    wordDictB[word]+=1

occurence = pd.DataFrame([wordDictA, wordDictB])
occurence

{'the', 'is', 'of', 'Data', '21st', 'for', 'science', 'data', 'century', 'Science', 'machine', 'key', 'job', 'sexiest', 'learning'}


,the,is,of,Data,21st,for,science,data,century,Science,machine,key,job,sexiest,learning
0,2,1,1,1,1,0,0,0,1,1,0,0,1,1,0
1,1,1,0,0,0,1,1,1,0,0,1,1,0,0,1


In [108]:
def tf(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)
    
#running our sentences through the tf function:
tfFirst = tf(wordDictA, first_sentence)
tfSecond = tf(wordDictB, second_sentence)

#Converting to dataframe for visualization
tf = pd.DataFrame([tfFirst, tfSecond])
tf

,the,is,of,Data,21st,for,science,data,century,Science,machine,key,job,sexiest,learning
0,0.200,0.100,0.1,0.1,0.1,0.000,0.000,0.000,0.1,0.1,0.000,0.000,0.1,0.1,0.000
1,0.125,0.125,0.0,0.0,0.0,0.125,0.125,0.125,0.0,0.0,0.125,0.125,0.0,0.0,0.125


In [109]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

set(stopwords.words('english'))

filtered_sentence = []
for word in wordDictA:
    if str(word) not in set(stopwords.words('english')):
        filtered_sentence.append(word)

filtered_sentence

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/todsavadt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Data',
 '21st',
 'science',
 'data',
 'century',
 'Science',
 'machine',
 'key',
 'job',
 'sexiest',
 'learning']

In [110]:
import math

def IDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / (float(val) + 1))
        
    return(idfDict)
    
#inputing our sentences in the log file
idfs = IDF([wordDictA, wordDictB])
idfs

{'the': 0.3010299956639812,
 'is': 0.3010299956639812,
 'of': 0.3010299956639812,
 'Data': 0.3010299956639812,
 '21st': 0.3010299956639812,
 'for': 0.3010299956639812,
 'science': 0.3010299956639812,
 'data': 0.3010299956639812,
 'century': 0.3010299956639812,
 'Science': 0.3010299956639812,
 'machine': 0.3010299956639812,
 'key': 0.3010299956639812,
 'job': 0.3010299956639812,
 'sexiest': 0.3010299956639812,
 'learning': 0.3010299956639812}

In [92]:
def TF_IDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return(tfidf)
#running our two sentences through the IDF:
idfFirst = TF_IDF(tfFirst, idfs)
idfSecond = TF_IDF(tfSecond, idfs)

#putting it in a dataframe
idf = pd.DataFrame([idfFirst, idfSecond])
idf

,the,is,of,Data,21st,for,science,data,century,Science,machine,key,job,sexiest,learning
0,0.060206,0.030103,0.030103,0.030103,0.030103,0.000000,0.000000,0.000000,0.030103,0.030103,0.000000,0.000000,0.030103,0.030103,0.000000
1,0.037629,0.037629,0.000000,0.000000,0.000000,0.037629,0.037629,0.037629,0.000000,0.000000,0.037629,0.037629,0.000000,0.000000,0.037629


In [98]:
import numpy as np

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    similarity = dot_product / (norm_a * norm_b)
    return round(similarity, 4)

# Example usage:
similarity_score = cosine_similarity(idf.iloc[0], idf.iloc[1])
print("Cosine Similarity A-B:", similarity_score)

Cosine Similarity A-B: 0.3062


### Using TfidfVectorizer from Sklearn

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Your sentences
firstV = "Data Science is the sexiest job of the 21st century"
secondV = "machine learning is the key for data science"

# Create a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transform the sentences into TF-IDF vectors (sparse matrix)
tfidf_matrix = vectorizer.fit_transform([firstV, secondV])

# Use cosine_similarity with the sparse matrix directly
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Print or use the similarity score
print("Cosine Similarity between the two sentences:", similarity_matrix[0, 1])

Cosine Similarity between the two sentences: 0.3528003588287327


In [127]:
import pandas as pd
from rank_bm25 import BM25Okapi
import numpy as np

df = pd.read_csv("./data/Friends dialogues.csv")
df.head()

,episode,Dialogue
0,1x01 - The One Where Monica Gets A Roommate.7...,there s nothing to tell it s just some guy i w...
1,1x02 - The One With The Sonogram At The End.,you guys don t understand for us kissing is as...
2,1x03 - The One With The Thumb.720p HDTV.TvR.,hi guys hey pheebs oh oh how d it go um not so...
3,1x04 - The One With George Stephanopoulos.720...,all right phoebe if i were omnipotent for a da...
4,1x05 - The One With The East German Laundry D...,let it go it s not a big deal not a big deal i...


In [124]:
corpus = df['Dialogue'].tolist()
# corpus
tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [133]:
def search(query):
    query=query.lower()
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    index = np.argsort(-doc_scores)[:5][0]
    return df.iloc[index][0]

query = "there is nothing"
episode = search(query)
print(episode)

 8x14 - The One With The Secret Closet.
